# Loading Libraries

In [507]:
# Importing Libs
import os
import numpy as np # Linear Algebra
import pandas as pd # Data Manipulation
import seaborn as sns # Data Viz
import matplotlib.pyplot as plt # Data Viz
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Perform OneHotEnconding
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#warnings
import warnings
warnings.filterwarnings("ignore")
# os.chdir("/content/drive/MyDrive/Midterm-project_I")

In [258]:
%%capture
!pip install xgboost==1.6.1

## 1. Load Weather Data

In [ ]:
dff=pd.read_csv('df_merged_final.csv',index_col=[0])


In [ ]:
dff.describe()

In [ ]:
#plot distributions
axes = dff.iloc[:,35:49].hist(layout=(5,6), alpha=0.5, label='x',figsize=(25, 15))
plt.tight_layout()
plt.show()

In [ ]:
# plot precipitation distribution
axes = dff.iloc[:,42:43].hist(layout=(1,1), alpha=0.5, label='x',figsize=(25, 15), bins=30)
plt.tight_layout()
plt.show()

## 2.Weather Feature Engineering

In [ ]:
# encode the weather data at origin
def newlabel_origin(df):
    
    if df['Origincloudcover']<40 and df['OriginprecipMM'] == 0:
        return "clear"
    if df['Origincloudcover']>=40 and df['OriginprecipMM'] == 0:
        return "cloudy"
    if 0<df['OriginprecipMM']<=10:
        return "lightrain"
    if 10<df['OriginprecipMM']<=25:
        return "heavyrain"
    if df['OriginprecipMM']>25:
        return "storm"

df['Weather_origin']=df.apply(newlabel_origin, axis=1)

# encode the weather data at destination
def newlabel_destin(df):
    
    if df['Destincloudcover']<40 and df['DestinprecipMM'] == 0:
        return "clear"
    if df['Destincloudcover']>=40 and df['DestinprecipMM'] == 0:
        return "cloudy"
    if 0<df['DestinprecipMM']<=10:
        return "lightrain"
    if 10<df['DestinprecipMM']<=25:
        return "heavyrain"
    if df['DestinprecipMM']>25:
        return "storm"

df['Weather_destin']=df.apply(newlabel_destin, axis=1)

In [ ]:
# encode the visibility 
def visibility_origin(df):
    
    if df['Originvisibility']<7:
        return "lowvisibility"
    if 7<=df['Originvisibility']<9:
        return "mediumvisbility"
    if df['Originvisibility']>=9:
        return "highvisibility"
    

df['Visibility_origin']=df.apply(visibility_origin, axis=1)

def visibility_destin(df):
    if df['Destinvisibility']<7:
        return "lowvisibility"
    if 7<=df['Destinvisibility']<9:
        return "mediumvisbility"
    if df['Destinvisibility']>=9:
        return "highvisibility"
    
df['Visibility_destin']=df.apply(visibility_destin, axis=1)

In [ ]:
#encode the wind speed
def windspeed_origin(df):
    
    if df['OriginwindgustKmph']<45:
        return "lowwindspeed"
    if df['OriginwindgustKmph']>=45:
        return "highwindspeed"

dff['windspeed_origin']=dff.apply(windspeed_origin, axis=1)


def windspeed_destin(df):
    
    if df['DestinwindgustKmph']<45:
        return "lowwindspeed"
    if df['DestinwindgustKmph']>=45:
        return "highwindspeed"
    
dff['windspeed_destin']=dff.apply(windspeed_destin, axis=1)

In [ ]:
dff[dff.isnull().any(axis=1)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,DestinwindchillC,DestinwindgustKmph,Destincloudcover,Destinhumidity,DestinprecipMM,DestintempC,Destinvisibility,DestinwindspeedKmph,windspeed_origin,windspeed_destin


In [ ]:
#remove redundant weather columns
colstodrop = ['OriginmaxtempC', 'OriginmintempC',
       'Origintotalsnow_cm', 'Originsunhour', 'OriginfeelslikeC',
       'OriginheatindexC', 'OriginwindchillC', 'OriginwindgustKmph',
       'Origincloudcover', 'Originhumidity', 'OriginprecipMM', 'OrigintempC',
       'Originvisibility', 'OriginwindspeedKmph', 'DestinmaxtempC',
       'DestinmintempC', 'Destintotalsnow_cm', 'Destinsunhour',
       'DestinfeelslikeC', 'DestinheatindexC', 'DestinwindchillC',
       'DestinwindgustKmph', 'Destincloudcover', 'Destinhumidity',
       'DestinprecipMM', 'DestintempC', 'Destinvisibility',
       'DestinwindspeedKmph']


dff.drop(colstodrop,axis=1,inplace=True)

In [ ]:
#Exporting CSV file with weather features to add more features
# dff.to_csv('df_merged_final_weathermodified.csv')

AttributeError: '_iLocIndexer' object has no attribute 'to_csv'

In [ ]:
#correlation matrix
dff.corr()

# Clean up the testingfile for final prediction

In [261]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("/content/drive/MyDrive/Midterm-project_I")

## 1.Load Data

In [262]:
df_original=pd.read_csv('df_merged_final_weathermodified.csv',index_col=[0] )

In [263]:
df_test=pd.read_csv('df_merged_final_weathermodified_test.csv',index_col=[0] ).drop('Unnamed: 0',axis=1)

In [264]:
df=df_original.copy()

In [265]:
df_test['fl_date'] = pd.to_datetime(df_test['fl_date'])

In [266]:
df_test['weekday']=df_test['fl_date'].dt.dayofweek

In [372]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,Weather_origin,Weather_destin,Visibility_origin,Visibility_destin,windspeed_origin,windspeed_destin,weekday,crs_arr_timeblk,crs_dep_timeblk
0,2020-01-01,WN,WN,WN,3719,WN,N941WN,3719,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Morning
1,2020-01-01,WN,WN,WN,4863,WN,N496WN,4863,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,RedEye
2,2020-01-01,WN,WN,WN,4924,WN,N7732A,4924,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Evening
3,2020-01-01,WN,WN,WN,5321,WN,N7863A,5321,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Afternoon
4,2020-01-01,WN,WN,WN,4887,WN,N246LV,4887,14027,PBI,...,883,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17765,2020-01-01,AS,AS,AS,70,AS,N644AS,70,12523,JNU,...,95,storm,storm,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,RedEye,Evening
17766,2020-01-01,AS,AS,AS,64,AS,N622AS,64,12523,JNU,...,123,storm,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon
17767,2020-01-01,AS,AS,AS,65,AS,N607AS,65,15841,WRG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
17768,2020-01-01,AS,AS,AS,64,AS,N622AS,64,14256,PSG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon


## 2.Data Clean

## 3.Exploratory Data Analysis

*is_delayed indicator

In [268]:
#new feature: is_delayed

result = []
for value in df['arr_delay']:
    if value > 0:
        result.append(1)
    elif value <= 0:
        result.append(0)

df['is_delayed'] = result  


### b.10 busiest airport-pairs with average delay by airline

In [269]:
businest_airport=['ATL', 'ORD', 'DEN', 'LAX', 'CLT', 'PHX', 'SEA', 'SFO', 'IAH', 'LGA']

*Above are the top ten businest airports with the most flights. both as departure and as desination airports

In [270]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,air_time,flights,distance,Weather_origin,Weather_destin,Visibility_origin,Visibility_destin,windspeed_origin,windspeed_destin,is_delayed
0,2018-01-01,DL,DL_CODESHARE,DL,3468,9E,N292PQ,3468,15380,TVC,...,35.0,1.0,207.0,lightrain,lightrain,lowvisibility,highvisibility,lowwindspeed,lowwindspeed,0
1,2018-01-01,DL,DL_CODESHARE,DL,4579,OO,N8903A,4579,15380,TVC,...,37.0,1.0,207.0,lightrain,lightrain,lowvisibility,highvisibility,lowwindspeed,lowwindspeed,0
2,2018-01-01,DL,DL_CODESHARE,DL,4669,OO,N931EV,4669,15380,TVC,...,32.0,1.0,207.0,lightrain,lightrain,lowvisibility,highvisibility,lowwindspeed,lowwindspeed,1
3,2018-01-01,DL,DL_CODESHARE,DL,3470,9E,N309PQ,3470,13871,OMA,...,88.0,1.0,651.0,clear,lightrain,highvisibility,highvisibility,lowwindspeed,lowwindspeed,1
4,2018-01-01,NK,NK,NK,342,NK,N637NK,342,13487,MSP,...,69.0,1.0,528.0,cloudy,lightrain,highvisibility,highvisibility,lowwindspeed,lowwindspeed,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715366,2019-01-31,HA,HA_CODESHARE,HA,684,EM,N804HC,684,12173,HNL,...,30.0,1.0,72.0,lightrain,lightrain,highvisibility,highvisibility,highwindspeed,lowwindspeed,1
715367,2019-01-31,G4,G4,G4,940,G4,234NV,940,14112,PIE,...,146.0,1.0,1224.0,lightrain,lightrain,lowvisibility,highvisibility,lowwindspeed,lowwindspeed,1
715368,2019-01-31,UA,UA,UA,174,UA,N36280,174,12016,GUM,...,24.0,1.0,129.0,clear,heavyrain,highvisibility,highvisibility,lowwindspeed,highwindspeed,0
715369,2019-01-31,UA,UA,UA,104,UA,N39726,104,12016,GUM,...,26.0,1.0,129.0,clear,heavyrain,highvisibility,highvisibility,lowwindspeed,highwindspeed,0


### b.delay by time

#### b.1. categorize arr_delay by day of week:
* are flights tends to delay during the weekdays or weekends?

#### b.2. creating time block desination

In [271]:
def arr_time(x):
    if x>=700 and x<=1200:
        return "Morning"
    elif x>1200 and x<=1800:
        return "Afternoon"
    elif x>1800 and x<=2300:
        return "Evening"
    elif (x>2300 and x<=2400) or (x<700):
        return "RedEye"
    

#add arrival/departure timeblock to testing file

In [272]:
df_test['crs_arr_timeblk'] = df_test['crs_arr_time'].apply(lambda x :arr_time(x))
df['crs_arr_timeblk'] = df['crs_arr_time'].apply(lambda x :arr_time(x))


In [273]:
df_test['crs_dep_timeblk'] = df_test['crs_dep_time'].apply(lambda x :arr_time(x))
df['crs_dep_timeblk'] = df['crs_dep_time'].apply(lambda x :arr_time(x))

In [274]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,Weather_origin,Weather_destin,Visibility_origin,Visibility_destin,windspeed_origin,windspeed_destin,weekday,crs_arr_timeblk,crs_dep_timeblk
0,2020-01-01,WN,WN,WN,3719,WN,N941WN,3719,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Morning
1,2020-01-01,WN,WN,WN,4863,WN,N496WN,4863,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,RedEye
2,2020-01-01,WN,WN,WN,4924,WN,N7732A,4924,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Evening
3,2020-01-01,WN,WN,WN,5321,WN,N7863A,5321,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Afternoon
4,2020-01-01,WN,WN,WN,4887,WN,N246LV,4887,14027,PBI,...,883,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17765,2020-01-01,AS,AS,AS,70,AS,N644AS,70,12523,JNU,...,95,storm,storm,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,RedEye,Evening
17766,2020-01-01,AS,AS,AS,64,AS,N622AS,64,12523,JNU,...,123,storm,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon
17767,2020-01-01,AS,AS,AS,65,AS,N607AS,65,15841,WRG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
17768,2020-01-01,AS,AS,AS,64,AS,N622AS,64,14256,PSG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon


In [ ]:

#arr

In [169]:
dfff=df[df['is_delayed']==1]

In [170]:
df_rounte_day_timeblk_airline_avg_arr=dfff.groupby(['origin','dest','crs_arr_timeblk','op_unique_carrier']).agg({'arr_delay':'mean',
                      }).reset_index()

In [171]:
df_rounte_day_timeblk_airline_avg_arr=df_rounte_day_timeblk_airline_avg_arr.rename(columns={'arr_delay':'route_day_timeblk_airline_avg_arr'})

In [172]:
#dep

In [173]:
df_rounte_day_timeblk_airline_avg_dep=dfff.groupby(['origin','dest','crs_dep_timeblk','op_unique_carrier']).agg({'dep_delay':'mean',
                      }).reset_index()

In [174]:
df_rounte_day_timeblk_airline_avg_dep=df_rounte_day_timeblk_airline_avg_dep.rename(columns={'dep_delay':'route_day_timeblk_airline_avg_dep'})

In [175]:
df_rounte_day_timeblk_airline_avg_arr

,origin,dest,crs_arr_timeblk,op_unique_carrier,route_day_timeblk_airline_avg_arr
0,ABE,ATL,Afternoon,9E,50.750000
1,ABE,ATL,Afternoon,EV,1.000000
2,ABE,ATL,Evening,9E,46.571429
3,ABE,ATL,Evening,EV,5.750000
4,ABE,ATL,Morning,DL,59.300000
...,...,...,...,...,...
23232,YKM,SEA,RedEye,QX,9.375000
23233,YNG,SFB,Evening,G4,9.000000
23234,YUM,PHX,Afternoon,OO,22.181818
23235,YUM,PHX,Evening,OO,23.857143


In [176]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,Weather_origin,Weather_destin,Visibility_origin,Visibility_destin,windspeed_origin,windspeed_destin,weekday,crs_arr_timeblk,crs_dep_timeblk
0,2020-01-01,WN,WN,WN,3719,WN,N941WN,3719,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Morning
1,2020-01-01,WN,WN,WN,4863,WN,N496WN,4863,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,RedEye
2,2020-01-01,WN,WN,WN,4924,WN,N7732A,4924,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Evening
3,2020-01-01,WN,WN,WN,5321,WN,N7863A,5321,13931,ORF,...,159,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Afternoon
4,2020-01-01,WN,WN,WN,4887,WN,N246LV,4887,14027,PBI,...,883,clear,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17765,2020-01-01,AS,AS,AS,70,AS,N644AS,70,12523,JNU,...,95,storm,storm,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,RedEye,Evening
17766,2020-01-01,AS,AS,AS,64,AS,N622AS,64,12523,JNU,...,123,storm,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon
17767,2020-01-01,AS,AS,AS,65,AS,N607AS,65,15841,WRG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning
17768,2020-01-01,AS,AS,AS,64,AS,N622AS,64,14256,PSG,...,31,heavyrain,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon


In [196]:
#For_testing_data

In [300]:
df_arr=df.groupby(['op_unique_carrier']).agg({'arr_delay':'mean',}).reset_index()

In [303]:
df_dep=df.groupby(['op_unique_carrier']).agg({'dep_delay':'mean',}).reset_index()

In [301]:
df_arr=df_arr.rename(columns={'arr_delay':'avg_arr'})

In [304]:
df_dep=df_dep.rename(columns={'dep_delay':'avg_dep'})

In [305]:
df_t=df_test.copy()

In [306]:
df_t=df_t.merge(df_arr,on=['op_unique_carrier'],how='left').drop_duplicates(keep='first')

In [312]:
df_t=df_t.merge(df_dep,on=['op_unique_carrier'],how='left').drop_duplicates(keep='first')

In [313]:
df_t

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,Weather_destin,Visibility_origin,Visibility_destin,windspeed_origin,windspeed_destin,weekday,crs_arr_timeblk,crs_dep_timeblk,avg_arr,avg_dep
0,2020-01-01,WN,WN,WN,3719,WN,N941WN,3719,13931,ORF,...,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Morning,-1.379324,6.408516
1,2020-01-01,WN,WN,WN,4863,WN,N496WN,4863,13931,ORF,...,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,RedEye,-1.379324,6.408516
2,2020-01-01,WN,WN,WN,4924,WN,N7732A,4924,13931,ORF,...,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Evening,-1.379324,6.408516
3,2020-01-01,WN,WN,WN,5321,WN,N7863A,5321,13931,ORF,...,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Evening,Afternoon,-1.379324,6.408516
4,2020-01-01,WN,WN,WN,4887,WN,N246LV,4887,14027,PBI,...,clear,highvisibility,highvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning,-1.379324,6.408516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17765,2020-01-01,AS,AS,AS,70,AS,N644AS,70,12523,JNU,...,storm,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,RedEye,Evening,-4.464361,-0.969753
17766,2020-01-01,AS,AS,AS,64,AS,N622AS,64,12523,JNU,...,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon,-4.464361,-0.969753
17767,2020-01-01,AS,AS,AS,65,AS,N607AS,65,15841,WRG,...,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Morning,Morning,-4.464361,-0.969753
17768,2020-01-01,AS,AS,AS,64,AS,N622AS,64,14256,PSG,...,heavyrain,lowvisibility,lowvisibility,lowwindspeed,lowwindspeed,2,Afternoon,Afternoon,-4.464361,-0.969753


# Regression


## Data Preprocessing

In [ ]:
df=df.merge(df.groupby('op_unique_carrier')['arr_delay'].agg(['count','sum','mean']),on='op_unique_carrier')

In [401]:
dff=df_t[(df['dest'].isin(airport))&(df_t['origin'].isin(airport))]

### ONEHOTENCODE

In [402]:
# # # df_final =pd.read_csv('df_finalversion_data_final.csv')
# import pickle

# pickle_out = open("dfffffff.pickle","wb")
# pickle.dump(df, pickle_out)
# pickle_out.close()
# pickle_in = open("dfffffff.pickle","rb")
# df = pickle.load(pickle_in)


In [403]:
#  df_airport =pd.read_csv('df_finalversion_data_final_top10airports.csv')
#  df_final_modeling=df_airport.copy()

In [404]:
# df_sample=df_final.sample(n=10000)
df_sample=dff.copy()

In [387]:
# X=df_sample.drop('arr_delay',axis=1)

In [388]:
# y=df['arr_delay']

In [405]:
X=df_sample.copy()

In [406]:
X.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'Weather_origin', 'Weather_destin', 'Visibility_origin',
       'Visibility_destin', 'windspeed_origin', 'windspeed_destin', 'weekday',
       'crs_arr_timeblk', 'crs_dep_timeblk', 'avg_arr', 'avg_dep'],
      dtype='object')

In [407]:
#Select features

In [408]:
X_tmp=X[['dest_airport_id','origin_airport_id','op_unique_carrier','dest','origin','crs_arr_time','crs_dep_time','crs_elapsed_time',
       'Weather_origin', 'Weather_destin','crs_arr_timeblk','crs_dep_timeblk','weekday',
       'Visibility_origin', 'Visibility_destin', 'windspeed_origin',
       'windspeed_destin']]
# 'route_day_timeblk_airline_avg_arr',
#         'route_day_timeblk_airline_avg_dep'
# X_tmp=X[['origin_airport_id','distance','op_unique_carrier','dest','origin','crs_dep_time','crs_arr_time',]]


In [409]:
# X_tmp.corr()

In [410]:
# #one-hot encoded
# enc=OneHotEncoder()
# X_encoded=enc.fit_transform(X_tmp)
X_encoded=pd.get_dummies(X_tmp)

(715371,)

In [412]:
X_encoded.shape

(3989, 173)

In [ ]:
# X_1 = enc.fit_transform(X_tmp)


In [ ]:
X_encoded.shape

(48967, 72)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=0)

In [ ]:
# # Feature Scaling
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler(with_mean=False)
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

### 2. LABELENCODER

In [ ]:
X=df_sample.drop('arr_delay',axis=1)

In [ ]:
y=df_sample['arr_delay']

In [ ]:
X_tmp=X[['dest_airport_id','origin_airport_id','dest','origin','crs_arr_time',
       'Weather_origin', 'Weather_destin','crs_arr_timeblk','crs_dep_timeblk','weekday',
       'Visibility_origin', 'Visibility_destin', 'windspeed_origin',
       'windspeed_destin','route_day_timeblk_airline_avg_arr',
        'route_day_timeblk_airline_avg_dep']]

# X_tmp=X[['origin_airport_id','distance','op_unique_carrier','dest','origin','crs_dep_time','crs_arr_time',]]


In [ ]:
enc=LabelEncoder()
x=X_tmp.apply(enc.fit_transform)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

## Modeling RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [ ]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)

RandomForestRegressor()

In [ ]:
y_pred = rfr.predict(X_test)

In [ ]:
rfr.score(X_train,y_train)

0.8594102354150114

In [ ]:
rfr.score(X_test,y_test)

0.17330105880624025

In [ ]:
r2_score(y_test,y_pred)

0.20464169015178602

In [ ]:
#evaluation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV 

In [ ]:
#Randomized Search CV

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [15,20,30]
# Minimum number of samples required to split a node
min_samples_split = [10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [5, 10]


In [ ]:
# Create the random grid
# n_estimators:Number of trees in random forest

random_grid = {'n_estimators': [200],
               'max_features': ['auto','sqrt'],
               'max_depth': [15,],
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [ ]:
# Random search of parameters, using 5 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rfr, 
                               param_distributions = random_grid,
                               scoring='neg_mean_squared_error', 
                               n_iter = 3, cv = 4, 
                               verbose=2, random_state=42, 
                               n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)


Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] END max_depth=15, max_features=auto, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=  31.3s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=  30.1s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=  35.2s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=  30.2s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=   4.6s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=   4.6s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, n_estimators=200; total time=   4.7s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, n_estimat

RandomizedSearchCV(cv=4, estimator=RandomForestRegressor(), n_iter=3, n_jobs=1,
                   param_distributions={'max_depth': [15],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [5, 10],
                                        'min_samples_split': [10, 15],
                                        'n_estimators': [200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [ ]:
rf_random.best_params_


{'max_depth': 15,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 15,
 'n_estimators': 200}

In [ ]:
# {'max_depth': 15,
#  'max_features': 'auto',
#  'min_samples_leaf': 5,
#  'min_samples_split': 10,
#  'n_estimators': 200}

In [ ]:
y_predict=rf_random.predict(X_test)


In [ ]:
r2_score(y_test,y_predict)

0.28172616828884467

## Modeling XGBoost


In [ ]:
import xgboost as xgb
### XGBoost

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

#define model
#model = xgb.XGBRegressor(tree_method = 'gpu_hist')
model = xgb.XGBRegressor(tree_method = 'hist')

In [ ]:
#parameter definition
params = {"n_estimators": [3, 50, 100, 200, 500,700, 1000],
'min_child_weight': [1, 5, 7.5, 10],
'gamma': [0.5, 1, 1.5, 2, 5],
'subsample': [0.6, 0.8, 1.0],
'colsample_bytree': [0.6, 0.8, 1.0],
'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
'learning_rate': [.01, .02, .05, .1, .15, .2]
}

In [ ]:
# Instantiate RandomizedSearchCV()
model_rs = RandomizedSearchCV(model, param_distributions=params, scoring= 'r2', n_jobs=-1, verbose=5, random_state=42,refit=True, n_iter = 100)

# Train the model on the training set
model_rs.fit(X_train, y_train)

# predicted values
y_pred = model_rs.predict(X_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
# Print the best parameters and highest accuracy
print("Best parameters found: ", model_rs.best_params_)
print("Best score found: ", model_rs.best_score_)
print("Feature Importance", model_rs.best_estimator_.feature_importances_)
model_rs.cv_results_


Best parameters found:  {'subsample': 1.0, 'n_estimators': 50, 'min_child_weight': 7.5, 'max_depth': 10, 'learning_rate': 0.15, 'gamma': 0.5, 'colsample_bytree': 0.8}
Best score found:  0.22814851365575511
Feature Importance [0.00201003 0.00279097 0.0016356  ... 0.00148325 0.0015096  0.00188325]


{'mean_fit_time': array([13.89533072,  1.00776219, 13.62248731,  0.24932423, 14.0658505 ,
         0.17250237,  2.1570118 ,  9.72917008,  1.32852354,  1.11078305,
         0.86715183, 13.67590666,  0.68448591,  3.9193253 , 10.14961567,
         0.20562625,  6.88461165,  1.35693631,  6.2186902 ,  8.27557974,
         2.98671336, 13.37214379, 31.82109809, 14.86592298,  0.11142211,
         5.91343803, 10.96651669,  1.5053268 ,  4.1162447 ,  8.64052663,
        14.00817409,  0.22803292,  8.47372432,  0.10665627,  6.60431399,
         7.04827461,  7.09629979,  0.29633927,  5.50490303,  0.10537143,
         1.48530784,  2.35665874,  1.17377644,  4.16569905, 28.85037155,
         5.38402967,  4.08577628,  1.83476663,  8.47113914,  1.35039077,
         1.35148525,  4.96567221, 11.86303401,  1.60857844,  6.90602121,
         0.13123426,  0.21040363,  5.21740541,  0.13616662,  5.67545428,
         9.40042229, 11.54734721,  7.65250149, 16.29976788,  3.79652858,
        14.60705261,  6.43084593, 

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# R^2 score
r2score = metrics.r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print ("R^2 score is", r2score)
print ("RMSE score is", rmse)

R^2 score is 0.2286069383185585
RMSE score is 41.997668248467235


# Testing

In [21]:
# df_testing=pd.read_csv('df_merged_final_weathermodified_test.csv',index_col=[0])

In [314]:
df_t.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'Weather_origin', 'Weather_destin', 'Visibility_origin',
       'Visibility_destin', 'windspeed_origin', 'windspeed_destin', 'weekday',
       'crs_arr_timeblk', 'crs_dep_timeblk', 'avg_arr', 'avg_dep'],
      dtype='object')

In [429]:
df_origin_busiest=df.groupby(['origin']).agg({'flights':'count',}).sort_values(by='flights',ascending=False).reset_index().head(55)

In [430]:
airport=list(df_origin_busiest.origin)

In [505]:
df_t_new=df_t[(df_t['origin'].isin(airport))&(df_t['dest'].isin(airport))]

In [506]:
df_t_new.to_csv('df_test_without_dummies.csv')

In [486]:
X_tmp=df_t_new[['distance','origin_airport_id','op_unique_carrier','dest','origin','crs_dep_time','crs_arr_time',
       'Weather_origin', 'Weather_destin','crs_arr_timeblk','crs_dep_timeblk',
       'Visibility_origin', 'Visibility_destin', 'windspeed_origin','weekday',
       'windspeed_destin','avg_arr',
        'avg_dep']]

In [501]:
X=pd.get_dummies(X_tmp).drop('origin_BUF',axis=1)

In [502]:
X.columns

Index(['distance', 'origin_airport_id', 'crs_dep_time', 'crs_arr_time',
       'weekday', 'avg_arr', 'avg_dep', 'op_unique_carrier_9E',
       'op_unique_carrier_AA', 'op_unique_carrier_AS',
       ...
       'crs_dep_timeblk_Morning', 'crs_dep_timeblk_RedEye',
       'Visibility_origin_highvisibility', 'Visibility_origin_lowvisibility',
       'Visibility_origin_mediumvisbility', 'Visibility_destin_highvisibility',
       'Visibility_destin_lowvisibility', 'Visibility_destin_mediumvisbility',
       'windspeed_origin_lowwindspeed', 'windspeed_destin_lowwindspeed'],
      dtype='object', length=162)

In [503]:
X.shape

(10519, 162)

In [366]:
X.columns

Index(['distance', 'origin_airport_id', 'crs_dep_time', 'crs_arr_time',
       'weekday', 'avg_arr', 'avg_dep', 'op_unique_carrier_9E',
       'op_unique_carrier_AA', 'op_unique_carrier_AS', 'op_unique_carrier_AX',
       'op_unique_carrier_B6', 'op_unique_carrier_C5', 'op_unique_carrier_CP',
       'op_unique_carrier_DL', 'op_unique_carrier_EV', 'op_unique_carrier_F9',
       'op_unique_carrier_G7', 'op_unique_carrier_MQ', 'op_unique_carrier_NK',
       'op_unique_carrier_OH', 'op_unique_carrier_OO', 'op_unique_carrier_QX',
       'op_unique_carrier_UA', 'op_unique_carrier_WN', 'op_unique_carrier_YV',
       'op_unique_carrier_YX', 'dest_ATL', 'dest_CLT', 'dest_DEN', 'dest_DTW',
       'dest_EWR', 'dest_IAH', 'dest_LAS', 'dest_LAX', 'dest_LGA', 'dest_MCO',
       'dest_MSP', 'dest_ORD', 'dest_PHX', 'dest_SEA', 'dest_SFO',
       'origin_ATL', 'origin_CLT', 'origin_DEN', 'origin_DTW', 'origin_EWR',
       'origin_IAH', 'origin_LAS', 'origin_LAX', 'origin_LGA', 'origin_MCO',
       'ori

In [504]:
X.to_csv('df_test_with_dummies.csv')

In [192]:
import pickle
pickle_out = open("df_test_with_dummies.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [193]:

pickle_in = open("df_test_with_dummies.pickle","rb")
ffff = pickle.load(pickle_in)

In [ ]:
# #one-hot encoded
# enc=OneHotEncoder()
# X = enc.fit_transform(X_tmp)


In [184]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [185]:
rfr = RandomForestRegressor(max_depth=15,max_features='auto',min_samples_leaf= 5,min_samples_split= 10,n_estimators= 200)

In [ ]:
y_predict=rfr.fit(X)

In [ ]:
y_predict.to_csv('final_submission.csv')